In [64]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pdb
import urllib.request
from datetime import datetime
import json
import os

In [2]:
base_offers = "http://www.cittc.net/english/supplyList.html?offset=0&max=515"
base_requests = "http://www.cittc.net/sites/english/demandList.html?offset=0&max=101"

HEADERS = {
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"
}

# set up selenium
driver_path = os.environ['CHROME_DRIVER'] # change path as needed
browser = webdriver.Chrome(executable_path = driver_path)

# Scrape requests

In [3]:
browser.get(base_requests)

In [4]:
def get_link_from_list_item(i):
    details_xpath = '//*[@id="list"]/ul/li[{}]/div[6]'.format(i)
    links = browser.find_element_by_xpath(details_xpath)
    return "http://www.cittc.net" + links.get_attribute("innerHTML").split('"')[1] # extract link without having to BS it
        
request_links = [get_link_from_list_item(i) for i in range(1, 102)]

In [5]:
len(request_links)

101

In [7]:
def get_html_of_request_detail(link):
    request = urllib.request.Request(link, headers=HEADERS)
    page_html = urllib.request.urlopen(request).read().decode('utf-8')
    return BeautifulSoup(page_html, "lxml")

def parse_request_detail_content(link):
#     print("processing %s" % link)
    detail = get_html_of_request_detail(link)
    row = {}
    row["sector"] = "".join([c for c in detail.find("h3").find("span").text if c.isalpha() or c.isspace()]).strip()
    row["title"] = detail.find("h3")["title"]
    row.update(parse_info_div(detail.find("div", id="info")))
    row.update(parse_description_div(detail))
    row.update(parse_contact_div(detail.find("div", id="contact")))
    return row
        
def parse_info_div(div):
    labels = div.find_all("span")
    content = [get_label_content(label) for label in labels]
    return dict(zip([label.text.strip()[:-1] for label in labels], content))
    
def get_label_content(label):
    content = str(label.next_sibling)
    if content != "\n":
        return content
    return label.next_sibling.next_sibling.text.strip()

def parse_description_div(detail):
    headers = [label.text[:-1].strip() for label in detail.find("div", id="text").find_all("span")]
    descriptions = [desc.text.strip() for desc in detail.find_all("div", class_="infoin word-break")]
    return dict(zip(headers, descriptions))

def parse_contact_div(div):
    info = div.find_all("output")
    return dict(tuple(pair.text.split(":")) for pair in info)
    
request_content = {link: parse_request_detail_content(link) for link in request_links}

In [8]:
request_content = pd.DataFrame(request_content).T
request_content

,Before,By,Country,Description of Demand,Email Address,Employer,Keywords,Market Prospect,Mode of Co-operation,Name,Name of Project Owner/ Holders,Organization Type,Publish,Secondary Field,Telephone,sector,title
http://www.cittc.net/anonym/demandDetail/232,2 Years,Caopanpan,China,The weight volume ratio of lightweight glass b...,cpp@chinatis.com,CITTC,Wine,NaN,Other Mode,Caopanpan,Beijing Collaborative Innovation Service League,Overseas Company/Agency,2014.11.01,Bio-agriculture,+86 10 82696139-812,Agriculture,Glass Wine Bottle Lightweight Technologies
http://www.cittc.net/anonym/demandDetail/233,2 Years,Caopanpan,China,Technical support for swash plate axial plunge...,cpp@chinatis.com,CITTC,"motor,hydraulic,Processing",NaN,Other Mode,Caopanpan,Beijing Collaborative Innovation Service League,Overseas Company/Agency,2014.11.01,Electronic Manufacturing,+86 10 82696139-812,Electronics,swash plate axial plunger motor
http://www.cittc.net/anonym/demandDetail/297edff84a568f12014a56a7dce80000,2 Years,caopanpan,China,Related Work Bases:Evaluate and test the exist...,cpp@chinatis.com,CITTC,Optimize FRB Fluidized Bed,NaN,Joint Development,caopanpan,"Guodian Inner Mongolia Jingyang Energy Co., Ltd.",State-owned Enterprise,2014.12.17,Other,+86 10-82696139-812,Life Sciences and New Energy,Optimize FRB Fluidized Bed Process
http://www.cittc.net/anonym/demandDetail/297edff84a568f12014a56d98b310001,2 Years,caopanpan,China,Related Work Bases：Optimize the temperature fi...,cpp@chinatis.com,CITTC,Monitor Temperature；Reduction Furnace,NaN,Joint Development,caopanpan,"Guodian Inner Mongolia Jingyang Energy Co., Ltd.",State-owned Enterprise,2014.12.17,Other,+86 10-82696139-812,Life Sciences and New Energy,Monitor Temperature of Reduction Furnace
http://www.cittc.net/anonym/demandDetail/297edff84a568f12014a56dd6c0b0002,2 Years,caopanpan,China,Related Work Bases：Compare the Company’s DCS w...,cpp@chinatis.com,CITTC,Optimize System Control ； Reduction Furnace,NaN,Joint Development,caopanpan,"Guodian Inner Mongolia Jingyang Energy Co., Ltd.",State-owned Enterprise,2014.12.17,Other,+86 10-82696139-812,Life Sciences and New Energy,Optimize System Control of Reduction Furnace
http://www.cittc.net/anonym/demandDetail/297edff84a568f12014a56e0bc9e0003,2 Years,caopanpan,China,Related Work Bases：Conduct statistical analysi...,cpp@chinatis.com,CITTC,Corrosion of Metallic Materials；Chlorosilane a...,NaN,Joint Development,caopanpan,"Guodian Inner Mongolia Jingyang Energy Co., Ltd.",State-owned Enterprise,2014.12.17,Other,+86 10-82696139-812,Life Sciences and New Energy,Research Corrosion of Metallic Materials by Ch...
http://www.cittc.net/anonym/demandDetail/297edff84a568f12014a56e4f0b10004,2 Years,caopanpan,China,Related Work Bases:Optimize the technological ...,cpp@chinatis.com,CITTC,Electronic Grade Polycrystalline Silicon,NaN,Joint Development,caopanpan,"Guodian Inner Mongolia Jingyang Energy Co., Ltd.",State-owned Enterprise,2014.12.17,Other,+86 10-82696139-812,Life Sciences and New Energy,Technology for Control of Electronic Grade Pol...
http://www.cittc.net/anonym/demandDetail/297edff84a568f12014a56e8cfde0005,2 Years,caopanpan,China,Related Work Bases:The Company has completed t...,cpp@chinatis.com,CITTC,Wind Turbine Unit ;Integration Technology ; El...,NaN,Joint Development,caopanpan,"Inner Mongolia Geoho Energy Technology Co., Ltd.",Private Enterprise,2014.12.17,Other,+86 10-82696139-812,Life Sciences and New Energy,Integration Technology for Electrical System o...
http://www.cittc.net/anonym/demandDetail/297edff84a568f12014a56ed7f5b0006,2 Years,caopanpan,China,Related Work Bases:The Company has completed t...,cpp@chinatis.com,CITTC,Structural Design ; Wind Turbine,NaN,Joint Development,caopanpan,"Inner Mongolia Geoho Energy Technology Co., Ltd.",Private Enterprise,2014.12.17,Other,+86 10-82696139-812,Life Sciences and New Energy,Structural Design of Wind Turbine
http://www.cittc.net/anonym/demandDetail/297edff84a568f12014a56f077340007,2 Years,caopanpan,Ch

# Scrape offers

In [71]:
def get_link_from_list_item(i):
    details_xpath = '//*[@id="list"]/ul/li[{}]/div[1]'.format(i)
    links = browser.find_element_by_xpath(details_xpath)
    return "http://www.cittc.net" + links.get_attribute("innerHTML").split('"')[3] # extract link without having to BS it


max_offers = 515
offer_links = []
offset = 0
max_val = 103 # evenly divides into 515

while offset < max_offers:  
    offers_link = "http://www.cittc.net/english/supplyList.html?offset={}&max={}".format(offset, max_val)
    print("processing %s" % offers_link)
    browser.get(offers_link)
    offer_links = offer_links + [get_link_from_list_item(i) for i in range(1, max_val+1)]
    print ("offers: %d" % len(offer_links))
    offset += max_val

processing http://www.cittc.net/english/supplyList.html?offset=0&max=103
offers: 103
processing http://www.cittc.net/english/supplyList.html?offset=103&max=103
offers: 206
processing http://www.cittc.net/english/supplyList.html?offset=206&max=103
offers: 309
processing http://www.cittc.net/english/supplyList.html?offset=309&max=103
offers: 412
processing http://www.cittc.net/english/supplyList.html?offset=412&max=103
offers: 515


In [72]:
offer_links[:10]

['http://www.cittc.net/anonym/supplyDetail/297edff850934b310150a24b20ee00ca',
 'http://www.cittc.net/anonym/supplyDetail/297edff850934b310150a24b20de00c9',
 'http://www.cittc.net/anonym/supplyDetail/297edff850934b310150a24b20bf00c8',
 'http://www.cittc.net/anonym/supplyDetail/297edff850934b310150a24b20af00c7',
 'http://www.cittc.net/anonym/supplyDetail/297edff850934b310150a24b20a000c6',
 'http://www.cittc.net/anonym/supplyDetail/297edff84b47e1c1014b7bb56ecb0010',
 'http://www.cittc.net/anonym/supplyDetail/297edff84b47e1c1014b7bb3afd7000f',
 'http://www.cittc.net/anonym/supplyDetail/297edff84b47e1c1014b7bb18752000e',
 'http://www.cittc.net/anonym/supplyDetail/297edff84b47e1c1014b7baf9f4a000d',
 'http://www.cittc.net/anonym/supplyDetail/297edff84b47e1c1014b7badb726000c']

In [73]:
def get_html_of_request_detail(link):
    request = urllib.request.Request(link, headers=HEADERS)
    page_html = urllib.request.urlopen(request).read().decode('utf-8')
    return BeautifulSoup(page_html, "lxml")

def parse_request_detail_content(link):
    detail = get_html_of_request_detail(link)
    row = {}
    row["sector"] = "".join([c for c in detail.find("h3").find("span").text if c.isalpha() or c.isspace()]).strip()
    row["title"] = detail.find("h3")["title"]
    row.update(parse_info_div(detail.find("div", id="info")))
    row.update(parse_description_div(detail))
    row.update(parse_contact_div(detail.find("div", id="contact")))
    return row
        
def parse_info_div(div):
    labels = div.find_all("span")
    content = [get_label_content(label) for label in labels]
    return dict(zip([label.text.strip()[:-1] for label in labels], content))
    
def get_label_content(label):
    content = str(label.next_sibling)
    if content != "\n":
        return content
    return label.next_sibling.next_sibling.text.strip()

def parse_description_div(detail):
    headers = [label.text[:-1].strip() for label in detail.find("div", id="text").find_all("span")]
    descriptions = [desc.text.strip() for desc in detail.find_all("div", class_="infoin word-break")]
    return dict(zip(headers, descriptions))

def parse_contact_div(div):
    info = div.find_all("output")
    return dict(tuple(pair.text.split(":")) for pair in info)
    
offer_content = {link: parse_request_detail_content(link) for link in offer_links}

In [74]:
offer_content_df = pd.DataFrame(offer_content).T

# Sanitize

In [68]:
# shared santization functions

def sanitize_keywords(keyword):
    if keyword != "No" and pd.notnull(keyword):
        return keyword.lower().replace(";", ",").replace(",", ", ").replace("  ", " ").replace(" ,", ",")

def sanitize_title(title):
    title = title.strip().replace("  ", " ").replace("“", "")
    return title

def sanitize_sector(sector):
    return sector.replace("   ", " ")

# add meta fields
def add_meta_fields(data, category):
    data["meta_organization"] = "CITTC"
    data["meta_category"] = category
    data["meta_collected_date"] = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
    return data

# write json
def write_json(data, file_name):
    with open(file_name, "w") as f:
        json_data = data.to_json(orient="records")
        f.write(json_data)

## Requests

In [15]:
request_content.columns

Index(['Before', 'By', 'Country', 'Description of Demand', 'Email Address',
       'Employer', 'Keywords', 'Market Prospect', 'Mode of Co-operation',
       'Name', 'Name of Project Owner/ Holders', 'Organization Type',
       'Publish', 'Secondary Field', 'Telephone ', 'sector', 'title'],
      dtype='object')

In [37]:
requests_column_map = {"index": "meta_base_url",
                       "title": "title",
                       "sector": "sector",
                       "Publish": "published",
                       "Before": "window",
                       "Country": "country",
                       "Description of Demand": "description",
                       "Secondary Field": "secondary_field",
                       "Keywords": "keywords",
                       "Mode of Co-operation": "cooperation_type",
#                        "Market Prospect": "market_prospect",
                       "Name of Project Owner/ Holders": "contact_organization",
                       "Organization Type": "contact_organization_type",
                       "Name": "contact_person",
                       "Employer": "contact_employer",
                       "Email Address": "contact_email",
                       "Telephone ": "contact_phone"
                      }


clean_requests = request_content.copy()
clean_requests.reset_index(inplace=True)
clean_requests.rename(columns=requests_column_map, inplace=True)
clean_requests = clean_requests[list(requests_column_map.values())]
clean_requests.sample()

,meta_base_url,title,sector,published,window,country,description,secondary_field,keywords,cooperation_type,contact_organization,contact_organization_type,contact_person,contact_employer,contact_email,contact_phone
87,http://www.cittc.net/anonym/demandDetail/908,design and manufacturing of large tube forming...,Electronics,2014.11.01,2 Years,China,Project required: design and manufacturing of ...,Electronic Manufacturing,No,Other Mode,Zhejiang Jiaxing Yada Stainless Steel Manufact...,Overseas Company/Agency,Caopanpan,CITTC,cpp@chinatis.com,+86 10 82696139-812


In [56]:
clean_requests["title"] = clean_requests["title"].apply(sanitize_title)
clean_requests["keywords"] = clean_requests["keywords"].apply(sanitize_keywords)
clean_requests["sector"] = clean_requests["sector"].apply(sanitize_sector)
clean_requests = add_meta_fields(clean_requests, "Technology Request")

In [57]:
clean_requests.sample()

,meta_base_url,title,sector,published,window,country,description,secondary_field,keywords,cooperation_type,contact_organization,contact_organization_type,contact_person,contact_employer,contact_email,contact_phone,meta_organization,meta_category,meta_collected_date
84,http://www.cittc.net/anonym/demandDetail/901,Chinese companies seeking for technology projects,Life Sciences and New Energy,2014.11.01,2 Years,China,"A Chinese company in central China, looks for ...",Biomass,energy,Other Mode,ITTN BUSINESS CENTER,Overseas Company/Agency,Caopanpan,CITTC,cpp@chinatis.com,+86 10 82696139-812,CITTC,Technology Request,11/08/2017 10:28:43


In [59]:
clean_requests["keywords"].value_counts()[:3]

electronic aluminium foil                  2
organic sulfur contained, coke oven gas    1
water flash evaporation                    1
Name: keywords, dtype: int64

### Write clean CSV/JSON

In [60]:
clean_requests.to_csv("cittc_technology_requests.csv", index=False, encoding="utf8")

In [69]:
write_json(clean_requests, "cittc_technology_requests.json")

## Offers

In [ ]:
offers_column_map = {"index": "meta_base_url",
                     "Publish": "published",
                     "Before": "window",
                     "Country": "country",
                     "Region": "region",
                     "Project Description": "description",
                     "Secondary Field": "secondary_field",
                     "Technology Readiness Level": "technology_readiness_level",
                     "Keywords": "keywords",
                     "Intellectual Property": "intellectual_property_type",
                     "Filing / Grant No": "patent_info",
                     "Implementation": "implementation",
                     "Market Prospect": "market_prospect",
                     "Mode of Co-operation": "cooperation_type",
                     "Name of Project Owner/Holder": "contact_organization",
                     "Organization Type": "contact_organization_type",
                     "Name": "contact_person",
                     "Employer": "contact_employer",
                     "E-mail Address": "contact_email",
                     "Telephone": "contact_phone"
                    }

clean_offers = offer_content.copy().reset_index()
clean_offers.rename(columns=column_map, inplace=True)


In [ ]:
clean_offers.columns

In [ ]:
pd.set_option("display.max_colwidth", 50)
clean_offers.sample(5)

In [ ]:
def sanitize_keywords(keyword):
    if keyword != "No":
        return keyword.lower().replace(";", ",").replace(",", ", ").replace("  ", " ")

clean_offers["keywords"] = clean_offers["keywords"].apply(sanitize_keywords)

In [ ]:
def sanitize_title(title):
    title = title.strip().replace("  ", " ").replace("“", "")
    return title

clean_offers["title"] = clean_offers["title"].apply(sanitize_title)

In [ ]:
from datetime import datetime

# add meta fields
clean_offers["meta_organization"] = "CITTC"
clean_offers["meta_category"] = "Technology Offer"
clean_offers["meta_collected_date"] = datetime.now().strftime("%m/%d/%Y %H:%M:%S")

In [ ]:
clean_offers.sample()

In [ ]:
sorted(column_map.values())

In [ ]:
clean_offers.to_csv("cittc_offers.csv", index=False, encoding="utf-8")